In [10]:
from nexusync import NexuSync

OPENAI_MODEL_YN = True
EMBEDDING_MODEL = "text-embedding-3-large"
LANGUAGE_MODEL = "gpt-4o-mini"
TEMPERATURE = 0.01
INPUT_DIRS = ["../sample_docs"]
CHROMA_DB_DIR = "chroma_db"
INDEX_PERSIST_DIR = "index_storage"
CHROMA_COLLECTION_NAME = "my_collection"
CHUNK_SIZE = 1024
CHUNK_OVERLAP = 20
RECURSIVE = True


ns = NexuSync(
    input_dirs=INPUT_DIRS,
    openai_model_yn=OPENAI_MODEL_YN,
    embedding_model=EMBEDDING_MODEL,
    language_model=LANGUAGE_MODEL,
    temperature=TEMPERATURE,
    chroma_db_dir=CHROMA_DB_DIR,
    index_persist_dir=INDEX_PERSIST_DIR,
    chroma_collection_name=CHROMA_COLLECTION_NAME,
    chunk_overlap=CHUNK_OVERLAP,
    chunk_size=CHUNK_SIZE,
    recursive=RECURSIVE,
)

2024-10-12 16:52:18,027 - nexusync.utils.embedding_models.set_embedding_model - INFO - Using OpenAI embedding model: text-embedding-3-large
2024-10-12 16:52:18,029 - nexusync.utils.embedding_models.set_language_model - INFO - Using OpenAI LLM model: gpt-4o-mini
2024-10-12 16:52:18,029 - nexusync.NexuSync - INFO - Vectors and Querier initialized successfully.
2024-10-12 16:52:18,185 - nexusync.core.indexer - INFO - Index already built. Loading from disk.


In [23]:
text_qa_template = """
Context Information:
{context_str}
Query: {query_str}
Instructions:
You are helping NHS doctors to review patients' medical records and give interperetations on the results.
Carefully read the context information and the query.
If the query is in the format [patient_id, summary_report], generate a summary report using the template below.
Use the available information from the context to fill in each section.
Include relevant dates and timeline information in each section.
If information for a section is not available, state "No information available" for that section.
Provide concise and accurate information based on the given context.
Adapt the template as needed to fit the patient's specific medical history and conditions.

Summary Report Template:

Patient Summary Report for {patient_id}
1. Demographics

Name: [First Name] [Last Name]
Date of Birth: [DOB]
Gender: [Gender]
Contact Information:

Address: [Address]
Phone: [Phone Number]
Email: [Email Address]



2. Past Medical History & Procedures

Chronic Conditions: [List of chronic conditions with diagnosis dates]
Major Illnesses: [List of major illnesses with dates]
Surgical Procedures: [List of surgical procedures with dates]
Other Significant Medical Events: [List with dates]
Your interpretation: [Your interpretation of the medical records]

3. Medication History
[List each current medication with the following information]

Name: [Medication Name]
Dosage: [Dosage]
Frequency: [Frequency]
Start Date: [Start Date]
Prescriber: [Prescriber Name]
Purpose: [Brief description of why the medication is prescribed]

[Include a brief list of significant past medications, if available]
4. Allergies and Adverse Reactions

Medication Allergies: [List or "No known medication allergies"]
Other Allergies: [List or "No known other allergies"]
Adverse Reactions: [List any significant adverse reactions to treatments or medications]

5. Social History & Occupation

Occupation: [Current or most recent occupation]
Smoking Status: [Current smoker, former smoker, never smoker]
Alcohol Use: [Description of alcohol use]
Recreational Drug Use: [If applicable]
Exercise Habits: [Brief description]
Diet: [Any significant dietary information]
Other Relevant Social Factors: [e.g., living situation, support system]
Your interpretation: [Your interpretation of the social history]

6. Physical Examination & Vital Signs
Most Recent Vital Signs (Date: [Date of most recent vital signs])

Blood Pressure: [BP]
Heart Rate: [HR]
Respiratory Rate: [RR]
Temperature: [Temp]
Oxygen Saturation: [O2 Sat]
Weight: [Weight]
Height: [Height]
BMI: [BMI]
Your interpretation: [Your interpretation of the vital signs]
[Include any significant physical examination findings]

7. Laboratory Results
[List most recent significant laboratory tests with dates, results, and normal ranges]

8. Imaging and Diagnostic Results
[List recent imaging studies and other diagnostic tests with dates and summary of results]

9. Treatment Plan and Interventions

Current Treatment Plans: [List current treatments or interventions]
Ongoing Therapies: [e.g., physical therapy, chemotherapy, dialysis]
Recent Changes in Management: [Any recent significant changes in treatment]
Your interpretation: [Your interpretation of the treatment plan]

10. Immunizations
[List relevant immunizations with dates]

11. Upcoming Appointments and Follow-ups
[List any scheduled appointments with dates, types, and locations]


Answer: [Generate the report based on the template above, filling in the available information from the context]

Answer: """

In [24]:
# Initialize the Chat Engine Once
ns.initialize_stream_chat(
    text_qa_template=text_qa_template, chat_mode="context", similarity_top_k=3
)


2024-10-13 08:02:24,048 - nexusync.core.chat_engine - INFO - Chat engine initialized


In [25]:
query = "[P000000, summary_report]"

print("```markdown")
for item in ns.start_chat_stream(query):
    if isinstance(item, str):
        # This is a token, print or process as needed
        print(item, end='', flush=True)
    else:
        # This is the final response with metadata
        print("```\n")
        print("Full response:")
        print("```markdown")
        print(item['response'])
        print("```\n")
        print("Metadata:", item['metadata'])
        break

```markdown
Here is the summary report for Patient ID P000000:

### Patient Demographics
- **First Name:** PatientP000000
- **Last Name:** Doe
- **Date of Birth:** November 1, 1966
- **Gender:** Male
- **Contact Information:**
  - **Address:** 139 Elm Street, Springfield, IL, 62704
  - **Phone:** +44 7911 671531
  - **Email:** patientP000000@example.com

### Emergency Contact
- **Name:** Spouse of PatientP000000
- **Relationship:** Spouse
- **Phone:** +44 7911 376933

### Medical History
- **Condition:** Leukemia
  - **Diagnosis Date:** July 8, 2018
  - **Stage:** IV
  - **Treatment:** Chemotherapy
  - **Notes:** Stage IV Leukemia diagnosed. Treatment plan includes Chemotherapy.
  - **Location Name:** Green Valley Cancer Institute

### Medications
1. **Name:** Imatinib
   - **Dosage:** 96 mg
   - **Frequency:** Every 3 weeks
   - **Start Date:** March 29, 2024
   - **Prescriber:** Dr. John Oncologist
   - **Location Name:** NHS Oncology Centre

2. **Name:** Nilotinib
   - **Dosage:** 1